In [0]:
import sys
import os

# 1. Configura Path para importar 'src'
repo_root = os.getcwd()
repo_root = '/'.join(repo_root.split('/')[:-1])
if repo_root not in sys.path:
    sys.path.append(repo_root)




In [0]:
%pip install -r ../requirements.txt

In [0]:
from src.utility.environment import Environment
from src.infrastructure.data_manager import DataManager
from src.infrastructure.ml_trainer import PySparkTrainer
from hyperopt import hp

In [0]:
# 1. Defina um caminho temporário dentro do SEU volume existente
tmp_path = Environment.MLFLOW_DFS_TMP

# 2. Cria a pasta se ela não existir (para evitar erros de caminho inválido)
if not os.path.exists(tmp_path):
    os.makedirs(tmp_path)

# 3. Define a variável de ambiente que o MLflow exige
os.environ['MLFLOW_DFS_TMP'] = tmp_path

In [0]:
# Importa múltiplos algoritmos
from pyspark.ml.classification import RandomForestClassifier, GBTClassifier, LogisticRegression

# 1. Carregar Dados
manager = DataManager(spark)
df_gold = manager.read_delta(f"{Environment.feature_store_path}/churn")

# 2. Instancia o Trainer Genérico
current_user = dbutils.notebook.entry_point.getDbutils().notebook().getContext().userName().get()
experiment_name = "churn_hyperopt_experiment"
full_experiment_path = f"/Users/{current_user}/{experiment_name}"
trainer = PySparkTrainer(full_experiment_path)

In [0]:
import pyspark.sql.functions as F

# 1. Calcule a proporção de cada classe
total = df_gold.count()
count_churn = df_gold.filter(F.col('churn') == 1).count()
count_nao_churn = total - count_churn

# 2. Defina os pesos (Estratégia: Inversamente proporcional à frequência)
# Se Churn é raro, ele ganha peso alto.
peso_churn = total / (2 * count_churn)
peso_nao_churn = total / (2 * count_nao_churn)

print(f"Peso Churn (1): {peso_churn:.2f}")
print(f"Peso Não-Churn (0): {peso_nao_churn:.2f}")



In [0]:
# 3. Adicione a coluna 'weight' ao DataFrame
df_gold = df_gold.withColumn(
    "weight",
    F.when(F.col("churn") == 1, F.lit(peso_churn))
     .otherwise(F.lit(peso_nao_churn))
)

In [0]:
# 4. Definindo as colunas utilizadas pelo modelo
features = ['recency', 'frequency', 'monetary', 'total_items_volume', 'payment_method_count', 'max_installments', 'avg_satisfaction', 'min_review_score', 'total_reviews_given']

In [0]:
# Definição do Espaço de Busca
space_gbt = {
    'maxDepth': hp.quniform('maxDepth', 3, 8, 1),
    'maxIter': hp.quniform('maxIter', 20, 60, 10),
    'stepSize': hp.uniform('stepSize', 0.01, 0.1),
    'weightCol': "weight"
}


# Note que passamos GBTClassifier (a classe), sem parênteses ()
best_params_gbt = trainer.tune(
    df=df_gold,
    estimator_cls=GBTClassifier, 
    search_space=space_gbt,
    feature_cols=features,
    max_evals=30,
    run_name="Tuning_GBT_V1"
)

In [0]:
# Definição do Espaço de Busca
space_rf = {
    'maxDepth': hp.quniform('maxDepth', 3, 10, 1),
    'maxBins': hp.quniform('maxBins', 16, 64, 1),
    'numTrees': hp.quniform('numTrees', 10, 40, 1),
    'weightCol': "weight"
}


# Note que passamos GBTClassifier (a classe), sem parênteses ()
best_params_rf = trainer.tune(
    df=df_gold,
    estimator_cls=RandomForestClassifier, 
    search_space=space_rf,
    feature_cols=features,
    max_evals=60,
    run_name="Tuning_RF_V1"
)

In [0]:
# Definição do Espaço de Busca
space_lr = {
    'maxIter': hp.quniform('maxIter', 60, 250, 1),
    'regParam': hp.quniform('regParam', 0, 1, 0.01),
    'elasticNetParam': hp.quniform('elasticNetParam', 0, 1, 0.01),
    'tol': hp.uniform('tol', 1e-7, 1e-5),
    'fitIntercept': hp.choice('fitIntercept', [True, False]),
    'weightCol': "weight"
}


# Note que passamos GBTClassifier (a classe), sem parênteses ()
best_params_lr = trainer.tune(
    df=df_gold,
    estimator_cls=LogisticRegression, 
    search_space=space_lr,
    feature_cols=features,
    max_evals=120,
    run_name="Tuning_LR_V1"
)

# Definindo um modelo para produção
Como visto, o modelo deve ser avaliado frente a métricas que capturam perspectivas de desbalanceamento, por isso foram otimizados para o AUC, ficando entre a Regressão Logistica e o RandomForest. 

Como Regressão Logistica é um modelo mais simples, decidi que seria melhor utiliza-lo. Por isso, ele foi adicionado como modelo de produção manualmente.